In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import numpy as np
import scipy.sparse
from scipy.sparse import csr_matrix
import json

Mounted at /content/drive


In [ ]:
!python3 -m pip install libpecos

     |████████████████████████████████| 15.2 MB 4.7 MB/s 
     |████████████████████████████████| 3.8 MB 82.0 MB/s 
     |████████████████████████████████| 1.2 MB 80.8 MB/s 
     |████████████████████████████████| 6.5 MB 82.0 MB/s 
     |████████████████████████████████| 67 kB 6.3 MB/s 
     |████████████████████████████████| 895 kB 90.2 MB/s 
     |████████████████████████████████| 596 kB 86.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
! unzip /content/drive/MyDrive/CS260c/Amazon13k_Bow/AmazonCat-13K.bow.zip
! unzip /content/drive/MyDrive/CS260c/Amazon13k_Raw/AmazonCat-13K.raw.zip
! gunzip -k /content/AmazonCat-13K.raw/trn.json.gz
! gunzip -k /content/AmazonCat-13K.raw/tst.json.gz
! unzip /content/drive/MyDrive/CS260c/260CDL_22Winter.zip

Archive:  /content/drive/MyDrive/CS260c/Amazon13k_Bow/AmazonCat-13K.bow.zip
   creating: AmazonCat-13K.bow/
  inflating: AmazonCat-13K.bow/test.txt  
  inflating: AmazonCat-13K.bow/train.txt  
  inflating: AmazonCat-13K.bow/Yf.txt  
  inflating: AmazonCat-13K.bow/Xf.txt  
Archive:  /content/drive/MyDrive/CS260c/Amazon13k_Raw/AmazonCat-13K.raw.zip
   creating: AmazonCat-13K.raw/
  inflating: AmazonCat-13K.raw/Yf.txt  
  inflating: AmazonCat-13K.raw/trn.json.gz  
  inflating: AmazonCat-13K.raw/tst.json.gz  
Archive:  /content/drive/MyDrive/CS260c/260CDL_22Winter.zip
   creating: 260CDL_22Winter/
  inflating: 260CDL_22Winter/test_explainer.sh  
  inflating: 260CDL_22Winter/.DS_Store  
  inflating: __MACOSX/260CDL_22Winter/._.DS_Store  
  inflating: 260CDL_22Winter/test_vectorizer.sh  
  inflating: __MACOSX/260CDL_22Winter/._test_vectorizer.sh  
  inflating: 260CDL_22Winter/toy_test_xlinear.py  
  inflating: __MACOSX/260CDL_22Winter/._toy_test_xlinear.py  
  inflating: 260CDL_22Winter/conf

In [ ]:
from pecos.xmc.xlinear.model import XLinearModel
from pecos.xmc import Indexer, LabelEmbeddingFactory
from pecos.utils import smat_util
from pecos.utils.featurization.text.preprocess import Preprocessor
from pecos.xmc.xtransformer.model import XTransformer
from pecos.xmc.xtransformer.module import MLProblemWithText

In [ ]:
# training from raw
# useful CSR explaination
# https://stackoverflow.com/questions/52299420/scipy-csr-matrix-understand-indptr

          # ins.  x dim  y dim
#training 1186239 203882 13330
#testing  306782  203882 13330
! rm '/content/drive/MyDrive/CS260c/train_corpus.txt'

corpus = []
x_vocab = []

y_indptr = [0]
y_indice = []
y_data = []
y_csrs = []
lenth = 1186239

with open("/content/AmazonCat-13K.bow/Xf.txt") as file:
  x_vocab = set([line.rstrip('\n') for line in file])

with open("/content/AmazonCat-13K.raw/trn.json") as myfile:
  for i in range(1,lenth+1):
    line = json.loads(next(myfile))
    x_line = [itm for itm in line["content"].lower().split() if itm in x_vocab]
    corpus.append(" ".join(x_line))
    y_indptr.append(y_indptr[-1] + len(line["target_ind"]) )
    y_indice = y_indice + line["target_ind"]
    if i % 100000 == 0 or i == lenth:
      print("current progress: ", i/(lenth+1.0) )
      y_data = [1] * len(y_indice)
      temp_csr = csr_matrix((y_data, y_indice, y_indptr), shape = (len(corpus),13330), dtype=np.float32 )
      y_csrs.append(temp_csr)
      with open('/content/drive/MyDrive/CS260c/train_corpus.txt', 'a') as f:
        for item in corpus:
          f.write("%s\n" % item)
      corpus = []
      y_indptr = [0]
      y_indice = []
      y_data = []

y_csr = y_csrs[0]
for item in y_csrs[1:]:
  y_csr = scipy.sparse.vstack((y_csr,item))
scipy.sparse.save_npz('/content/drive/MyDrive/CS260c/train_y.npz', y_csr)

print(y_csr.shape)
print(sum(1 for line in open('/content/drive/MyDrive/CS260c/train_corpus.txt')))

rm: cannot remove '/content/drive/MyDrive/CS260c/train_corpus.txt': No such file or directory
current progress:  0.08429997302400863
current progress:  0.16859994604801726
current progress:  0.2528999190720259
current progress:  0.3371998920960345
current progress:  0.42149986512004317
current progress:  0.5057998381440518
current progress:  0.5900998111680604
current progress:  0.674399784192069
current progress:  0.7586997572160777
current progress:  0.8429997302400863
current progress:  0.9272997032640949
current progress:  0.9999991570002698
(1186239, 13330)
1186239


In [ ]:
# loading vectorizer model 
! python3 -m pecos.utils.featurization.text.preprocess build \
  --text-pos 0 \
  --input-text-path /content/drive/MyDrive/CS260c/train_corpus.txt \
  --vectorizer-config-path /content/260CDL_22Winter/tfidf-model/config.json \
  --output-model-folder /content/tfidf-model

In [ ]:
! python3 -m pecos.utils.featurization.text.preprocess run \
    --input-preprocessor-folder /content/tfidf-model \
    --input-text-path /content/drive/MyDrive/CS260c/train_corpus.txt \
    --output-inst-path /content/x_trn.npz \
    --text-pos 0

In [ ]:
!cp /content/drive/MyDrive/CS260c/train_corpus.txt /content/train_corpus.txt
!cp /content/drive/MyDrive/CS260c/train_y.npz /content/train_y.npz

In [ ]:
# xlm = XLinearModel.load("/content/260CDL_22Winter/model", is_predict_only=False)
xlm = XLinearModel.load("/content/260CDL_22Winter/model", is_predict_only=True) # for faster prediction

In [ ]:
from pecos.utils.featurization.text.vectorizers import Vectorizer, vectorizer_dict
vectorizer = Vectorizer.load("/content/260CDL_22Winter/tfidf-model")

x_csr = vectorizer.predict(["walking to catch up to her people fleeing shoshoni she encounters"])
print(x_csr)
# print(xlm.predict(x_csr))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


  (0, 216297)	0.31869206
  (0, 325375)	0.33668044
  (0, 358791)	0.42867258
  (0, 418876)	0.169459
  (0, 632339)	0.20349121
  (0, 740100)	0.1829013
  (0, 743608)	0.6015959
  (0, 822284)	0.1499112
  (0, 850885)	0.1441928
  (0, 873034)	0.30423844


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [ ]:
! pip install lime
import lime
from lime.lime_text import LimeTextExplainer

     |████████████████████████████████| 275 kB 5.2 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=d288c3a8777d42310fef8ed7c894950ce55ef6f9b375995fc66abcf9b8ba1a26
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime


In [ ]:
# global timetime
# timetime = 0
# import time
class Prob_wrapper(object):
  def __init__(self, input_labels):
    self.labels = input_labels

  def pred_proba(self, sentence):
    # print(sentence)
    # s = time.time()
    tmp = vectorizer.predict(sentence)
    tmp.sort_indices()
    # print(tmp)
    y_pred = xlm.predict(tmp) 
    # global timetime
    # timetime += (time.time() - s)
    return y_pred[:,self.labels].toarray()

# x_csr = vectorizer.predict(["walking to catch up to her people fleeing shoshoni she encounters"])
# text = "walking to catch up to her people fleeing shoshoni she encounters"
# prob_tool = Prob_wrapper([1471, 7083])
# prob_tool.pred_proba([text])

In [ ]:
          # ins.  x dim  y dim
#training 1186239 203882 13330
#testing  306782  203882 13330
num_instance = 1186239
with open("/content/AmazonCat-13K.bow/Xf.txt", "r") as f:
  lines = f.read()
  x_feat = lines.splitlines()
x_feat_dict = {k: v for v, k in enumerate(x_feat)}
# scores_all = csr_matrix((num_instance, 203882))

In [ ]:
from lime.lime_text import IndexedString, TextDomainMapper
import sklearn
from sklearn.linear_model import Ridge
import numpy as np
import scipy as sp
from functools import partial
import json

# classifier_fn = Prob_wrapper([1471, 7083]).pred_proba
class Lime_Express(object):
  def __init__(self):
    self.RandState = np.random.RandomState()

  def feature_selection(self, data, labels, weights, num_features):
    clf = Ridge(alpha=0.01, fit_intercept=True, random_state=self.RandState)
    clf.fit(data, labels, sample_weight=weights)
    coef = clf.coef_

    if sp.sparse.issparse(data):
      coef = sp.sparse.csr_matrix(clf.coef_)
      weighted_data = coef.multiply(data[0])
      # Note: most efficient to slice the data before reversing
      sdata = len(weighted_data.data)
      argsort_data = np.abs(weighted_data.data).argsort()
      # Edge case where data is more sparse than requested number of feature importances
      # In that case, we just pad with zero-valued features
      if sdata < num_features:
          nnz_indexes = argsort_data[::-1]
          indices = weighted_data.indices[nnz_indexes]
          num_to_pad = num_features - sdata
          indices = np.concatenate((indices, np.zeros(num_to_pad, dtype=indices.dtype)))
          indices_set = set(indices)
          pad_counter = 0
          for i in range(data.shape[1]):
              if i not in indices_set:
                  indices[pad_counter + sdata] = i
                  pad_counter += 1
                  if pad_counter >= num_to_pad:
                      break
      else:
          nnz_indexes = argsort_data[sdata - num_features:sdata][::-1]
          indices = weighted_data.indices[nnz_indexes]
      return indices  
    else:
      weighted_data = coef * data[0]
      feature_weights = sorted(
          zip(range(data.shape[1]), weighted_data),
          key=lambda x: np.abs(x[1]),
          reverse=True)
      return np.array([x[0] for x in feature_weights[:num_features]])     

  def distance_fn(self, x):
      return sklearn.metrics.pairwise.pairwise_distances(
          x, x[0], metric="cosine").ravel() * 100

  def __data_labels_distances(self, indexed_string,
                              classifier_fn,
                              num_samples,
                              distance_metric='cosine'):
    doc_size = indexed_string.num_words()
    sample = np.random.randint(1, doc_size + 1, num_samples - 1)
    data = np.ones((num_samples, doc_size))
    data[0] = np.ones(doc_size)
    features_range = range(doc_size)
    inverse_data = [indexed_string.raw_string()]
    for i, size in enumerate(sample, start=1):
      inactive = np.random.choice(features_range, size, replace=False)
      data[i, inactive] = 0
      inverse_data.append(indexed_string.inverse_removing(inactive))
    labels = classifier_fn(inverse_data)
    distances = self.distance_fn(sp.sparse.csr_matrix(data))
    return data, labels, distances

  # explain_instance_with_data(data, yss, distances, label, num_features)
  def explain_instance_with_data(self, 
                                  neighborhood_data,
                                  neighborhood_labels,
                                  distances,
                                  label,
                                  num_features,):
    
    def kernel(d, kernel_width):
      return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2))

    kernel_width = 25
    kernel_fn = partial(kernel, kernel_width=kernel_width)

    weights = kernel_fn(distances)
    labels_column = neighborhood_labels[:, label]
    used_features = self.feature_selection(neighborhood_data, labels_column, weights, num_features)
    # model_regressor = Ridge(alpha=1, fit_intercept=True, random_state=np.random.RandomState())
    easy_model = Ridge(alpha=1, fit_intercept=True, random_state=np.random.RandomState())
    easy_model.fit(neighborhood_data[:, used_features],
                    labels_column, sample_weight=weights)
    return zip(used_features, easy_model.coef_)
  
  def explain(self, text_instance, labels, classifier_fn, num_samples, num_features):
  # text_instance = "people fleeing shoshoni she encounters"
    indexed_string = (IndexedString(text_instance, bow=True, split_expression = r'\W+', mask_string=None))
    domain_mapper = TextDomainMapper(indexed_string)
    data, yss, distances = self.__data_labels_distances(indexed_string, classifier_fn, num_samples)
    # labels = list(range(2)) #TODO modefy！！！
    local_pred = {}
    for label in labels:
      local_pred[label] = self.explain_instance_with_data(data, yss, distances, label, num_features)
    return local_pred
      

test_instance = "set in the northern plains in the years just before the blackfeet tribe acquired this is story about an resourceful blackfeet indian spotted flower is able to elude capture and survive hunger and miles of walking to catch up to her people fleeing shoshoni she encounters riderless wounded in buffalo overcoming her fear takes the animal she calls or elk that works like to her people so they can use it to rescue her two captured girl friends accurate background information has been carefully integrated into credible and consciously piece of historical journala fictitious account of the coming of the horse to the blackfeet indians is told in spotted flower and the the story centers around blackfeet girl of the spotted who "
lime_exp = Lime_Express()
explainer = LimeTextExplainer()
test_labels = list(range(2))
labels = [1471,7083]
prob_tool = Prob_wrapper(labels)
# pred_test = lime_exp.explain(test_instance, test_labels, prob_tool.pred_proba, 10000, 10)
exp = explainer.explain_instance(test_instance, prob_tool.pred_proba, num_samples=10000, num_features=10, top_labels = len(labels))
pred_test = exp.local_exp

main_dict = {}
for lb in test_labels:
  lb_dict = {}
  for itema,itemb in pred_test[lb]:
    # print(lb, itema, itemb)
    lb_dict[ x_feat_dict[test_instance.split()[itema]] ] = itemb
    print(labels[lb], x_feat_dict[test_instance.split()[itema]],itemb)
  feat_sort = dict(sorted(lb_dict.items(), key=lambda item: item[1], reverse=True) )
  main_dict[labels[lb]] = feat_sort
  print("")

ls_tt = [main_dict, main_dict]
with open('test.jsonl', 'w') as f:
  for ob in ls_tt:
    json.dump(ob, f)
    f.write('\n')
    # json.dump(main_dict, f)

data = []
def keystoint(x):
    return {int(k): v for k, v in x.items()}
with open('test.jsonl') as f:
  for line in f:
    data.append(json.loads(line, object_hook=keystoint))
print(data)

# (json.loads(j, object_hook=keystoint))

1471 36823 0.07919903412132001
1471 16990 0.06138835162413465
1471 12751 0.056292187167068575
1471 16773 0.055427279533336656
1471 181055 0.054356260712621216
1471 137571 0.05301980429994468
1471 86710 0.039424443040888116
1471 36822 0.03508712043957297
1471 200180 -0.02991980266416263
1471 183349 -0.023726492221345152

7083 99564 -0.15467825547701655
7083 16773 -0.1125288274947338
7083 96591 0.0740223722952266
7083 200401 0.06587592848187485
7083 137571 -0.06356111277735713
7083 195611 0.054577890120266676
7083 16990 0.05058099248147459
7083 12751 0.04986921098864872
7083 191281 0.04324110768990051
7083 12475 0.039026368835514295

[{1471: {36823: 0.07919903412132001, 16990: 0.06138835162413465, 12751: 0.056292187167068575, 16773: 0.055427279533336656, 181055: 0.054356260712621216, 137571: 0.05301980429994468, 86710: 0.039424443040888116, 36822: 0.03508712043957297, 183349: -0.023726492221345152, 200180: -0.02991980266416263}, 7083: {96591: 0.0740223722952266, 200401: 0.065875928481874

In [ ]:
# global timetime
# timetime = 0
explainer = LimeTextExplainer()
scores_all = csr_matrix((1, 203882))
timea = 0
timeb = 0
lime_expr = Lime_Express()
json_txt = ""

# y_csr = scipy.sparse.load_npz('/content/drive/MyDrive/CS260c/train_y.npz')
y_csr = scipy.sparse.load_npz('/content/train_y.npz')
idx_cnter = 0
# with open("/content/drive/MyDrive/CS260c/train_corpus.txt") as f:
with open("/content/train_corpus.txt") as f:
  idx_cnter = 1186000
  for _ in range(idx_cnter): # to be used when break
    next(f)

  for line in f:
    scores = np.zeros(203882)
    txt = line
    lbl = y_csr[idx_cnter,:].indices
    prob_tool = Prob_wrapper(lbl)

    if len(txt) > 1 :
      # sa = time.time()
      num_sps = min(len(txt), 100)
      #LIME: 
      # exp = explainer.explain_instance(txt, prob_tool.pred_proba, num_samples=num_sps, num_features=100, top_labels = len(lbl))
      #MY_LIME: 
      reind_labels = list(range(len(lbl)))
      explain_pairs = lime_expr.explain(txt, reind_labels, prob_tool.pred_proba, num_sps, 100)
      # ea = time.time()
      # timea += ea - sa

      # sb = time.time()
      #LIME: 
      # for itm in exp.local_exp:
      #   for idx, score in exp.local_exp[itm]:
      #MY_LIME: 
      label_dict = {}
      for itm in explain_pairs:
        feat_dict = {}
        for idx, score in explain_pairs[itm]:
          f_index = x_feat_dict[txt.split()[idx]]
          feat_dict[ str(f_index) ] = score
        feat_sort = dict(sorted(feat_dict.items(), key=lambda item: item[1], reverse=True) )
        label_dict[ str(lbl[itm]) ] = feat_sort
      # eb = time.time()
      # timeb += eb - sb
      json_txt += json.dumps(label_dict)
      json_txt += "\n"
    else:
      json_txt += json.dumps({"-1":0})
      json_txt += "\n"
      print("omitting long case: ", idx_cnter)

    # scores = scipy.sparse.csr_matrix(scores)
    # scores_all = scipy.sparse.vstack((scores_all,scores))

    idx_cnter += 1
    # if idx_cnter % 1000 == 0:
    if idx_cnter == 1186239:
      name = "/content/drive/MyDrive/CS260c/Xlr_feat_scores/scores" + str(idx_cnter) + ".jsonl"
      with open(name, "a") as f:
        f.write(json_txt)
      json_txt = ""
      # name = '/content/drive/MyDrive/CS260c/tmp/train_impo_mtx_'+str(idx_cnter)+'.npz'
      # scipy.sparse.save_npz(name, scores_all)
      print("current progress: ", idx_cnter/num_instance)
# print(scores_all)


current progress:  1.0


In [ ]:
# time: pred lime-expain other
# total 78 sec 
# 30.36753273010254 75.35491013526917 1.4491801261901855
# 30.87858748435974 76.01335167884827 1.3487579822540283
print(timetime, timea, timeb)

#lime time 400: 15.802817821502686 43.736669301986694 5.115905046463013

15.568015336990356 40.097657680511475 4.675274133682251


In [ ]:
# create a new corpus with importan feature removed
from itertools import islice
! rm /content/removed_train_corpus.txt
! touch /content/removed_train_corpus.txt

x_feat_dict_inv = {v: k for k, v in x_feat_dict.items()}
subfile_index = list(range(1000, 1187000, 1000)) + [1186239]

with open("/content/train_corpus.txt", 'r') as corpus, open("/content/removed_train_corpus.txt", 'a') as new_corpus:
  for subfile_id in subfile_index:
    print("current progress: ", subfile_id / 1186239)
    subfile_name = "/content/drive/MyDrive/CS260c/Xlr_feat_scores/scores"+str(subfile_id)+".jsonl"
    with open(subfile_name, 'r') as json_file:
        json_list = list(json_file)

    for json_str in json_list:
      flag = 0
      result = json.loads(json_str)
      important_feat = set()
      for keys, value in result.items():
        try:
          n_items = islice(value.items(), 5)
          l = [nword[1] for nword in n_items]
          important_feat.update(l)
          # important_feat.add( x_feat_dict_inv[ int(next(iter(value))) ] )
        except:
          flag = 1
      if flag == 0:
        stopwords = list(important_feat)
        query_ls = next(corpus).split()
        removed_ls  = [word for word in query_ls if word.lower() not in stopwords]
        removed = ' '.join(removed_ls)
        new_corpus.write("%s\n" % removed)
      else: 
        new_corpus.write("%s" % next(corpus) )

In [ ]:
! rm /content/drive/MyDrive/CS260c/removed_train_corpus.txt
! cp /content/removed_train_corpus.txt /content/drive/MyDrive/CS260c/removed_train_corpus.txt

In [ ]:
! python3 -m pecos.utils.featurization.text.preprocess run \
    --input-preprocessor-folder /content/260CDL_22Winter/tfidf-model \
    --input-text-path /content/drive/MyDrive/CS260c/removed_train_corpus.txt \
    --output-inst-path /content/removed_x_trn.npz \
    --text-pos 0

# ! python3 -m pecos.utils.featurization.text.preprocess run \
#     --input-preprocessor-folder /content/260CDL_22Winter/tfidf-model \
#     --input-text-path /content/drive/MyDrive/CS260c/test_corpus.txt \
#     --output-inst-path /content/x_tst.npz \
#     --text-pos 0

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [ ]:
X = XLinearModel.load_feature_matrix("/content/removed_x_trn.npz")
Y = XLinearModel.load_label_matrix("/content/train_y.npz", for_training=True)

# # construct label feature for clustering
label_feat = LabelEmbeddingFactory.create(Y, X, method="pifa")

# # generate label indexing
cluster_chain = Indexer.gen(label_feat, indexer_type="hierarchicalkmeans")

xlm = XLinearModel.train(X, Y, C=cluster_chain)
print("chain length is ", len(cluster_chain))

xlm.save("new_model")

chain length is  3


In [ ]:
# ! python3 -m pecos.utils.featurization.text.preprocess run \
#     --input-preprocessor-folder /content/260CDL_22Winter/tfidf-model \
#     --input-text-path /content/drive/MyDrive/CS260c/train_corpus.txt \
#     --output-inst-path /content/x_trn.npz \
#     --text-pos 0

X = XLinearModel.load_feature_matrix("/content/x_trn.npz")
Y = XLinearModel.load_label_matrix("/content/train_y.npz", for_training=True)

# # construct label feature for clustering
label_feat = LabelEmbeddingFactory.create(Y, X, method="pifa")

# # generate label indexing
cluster_chain = Indexer.gen(label_feat, indexer_type="hierarchicalkmeans")

xlm = XLinearModel.train(X, Y, C=cluster_chain)
print("chain length is ", len(cluster_chain))

xlm.save("old_model")

chain length is  3


In [ ]:
# xlm = XLinearModel.load("/content/260CDL_22Winter/model", is_predict_only=True) # for faster prediction
xlm = XLinearModel.load("/content/old_model", is_predict_only=True) # for faster prediction
xlm_new = XLinearModel.load("/content/new_model", is_predict_only=True) # for faster prediction
Xt = XLinearModel.load_feature_matrix("/content/x_tst.npz")

Y_pred = xlm.predict(Xt)
Y_pred_new = xlm_new.predict(Xt)

In [ ]:
from pecos.utils import smat_util
# load test labels with shape=(Nt, L)

Yt = XLinearModel.load_label_matrix("/content/drive/MyDrive/CS260c/test_y.npz")
metric = smat_util.Metrics.generate(Yt, Y_pred, topk=10)
print(metric)

prec   = 90.25 82.55 74.69 67.43 60.20 53.70 48.35 43.93 40.23 37.11
recall = 25.46 44.06 56.64 65.53 70.95 74.33 76.78 78.65 80.17 81.42


In [ ]:
metric = smat_util.Metrics.generate(Yt, Y_pred_new, topk=10)
print(metric)

prec   = 90.25 82.55 74.69 67.43 60.20 53.70 48.35 43.93 40.23 37.11
recall = 25.46 44.06 56.64 65.53 70.95 74.33 76.78 78.65 80.17 81.42


In [ ]:
! python3 -m pecos.utils.featurization.text.preprocess run \
    --input-preprocessor-folder /content/260CDL_22Winter/tfidf-model \
    --input-text-path /content/drive/MyDrive/CS260c/train_corpus.txt \
    --output-inst-path /content/x_trn.npz \
    --text-pos 0

xlm = XLinearModel.load("/content/260CDL_22Winter/model", is_predict_only=True) # for faster prediction
xlm_new = XLinearModel.load("/content/new_model", is_predict_only=True) # for faster prediction
Xtr = XLinearModel.load_feature_matrix("/content/x_trn.npz")

Y_pred = xlm.predict(Xtr)
Y_pred_new = xlm_new.predict(Xtr)

from pecos.utils import smat_util
# load test labels with shape=(Nt, L)

Y_tr = XLinearModel.load_label_matrix("/content/train_y.npz")
metric = smat_util.Metrics.generate(Y_tr, Y_pred, topk=10)
print(metric)
metric = smat_util.Metrics.generate(Y_tr, Y_pred_new, topk=10)
print(metric)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
prec   = 89.66 83.31 76.22 69.56 62.74 56.36 50.99 46.51 42.72 39.50
recall = 25.42 44.60 57.84 67.40 73.56 77.42 80.18 82.33 84.01 85.43
prec   = 94.82 89.61 82.90 76.42 69.08 62.05 56.10 51.09 46.86 43.26
recall = 27.33 48.55 63.28 7

In [ ]:
          # ins.  x dim  y dim
#training 1186239 203882 13330
#testing  306782  203882 13330
# ! rm /content/drive/MyDrive/CS260c/test_corpus.txt
# ! touch /content/drive/MyDrive/CS260c/test_corpus.txt

corpus = []
x_vocab = []

y_indptr = [0]
y_indice = []
y_data = []
y_csrs = []
lenth = 306782

with open("/content/AmazonCat-13K.bow/Xf.txt") as file:
  x_vocab = set([line.rstrip('\n') for line in file])

with open("/content/AmazonCat-13K.raw/tst.json") as myfile:
  for i in range(1,lenth+1):
    line = json.loads(next(myfile))
    x_line = [itm for itm in line["content"].lower().split() if itm in x_vocab]
    corpus.append(" ".join(x_line))
    y_indptr.append(y_indptr[-1] + len(line["target_ind"]) )
    y_indice = y_indice + line["target_ind"]
    if i % 30000 == 0 or i == lenth:
      print("current progress: ", i/(lenth+1.0) )
      y_data = [1] * len(y_indice)
      temp_csr = csr_matrix((y_data, y_indice, y_indptr), shape = (len(corpus),13330), dtype=np.float32 )
      y_csrs.append(temp_csr)

      # with open('/content/drive/MyDrive/CS260c/test_corpus.txt', 'a') as f:
      #   print()
        # for item in corpus:
        #   print(item)
        #   f.write("%s\n" % item)

      corpus = []
      y_indptr = [0]
      y_indice = []
      y_data = []

y_csr = y_csrs[0]
for item in y_csrs[1:]:
  y_csr = scipy.sparse.vstack((y_csr,item))
scipy.sparse.save_npz('/content/drive/MyDrive/CS260c/test_y.npz', y_csr)

current progress:  0.09778899091540275
current progress:  0.1955779818308055
current progress:  0.2933669727462082
current progress:  0.391155963661611
current progress:  0.4889449545770137
current progress:  0.5867339454924164
current progress:  0.6845229364078192
current progress:  0.782311927323222
current progress:  0.8801009182386247
current progress:  0.9778899091540274
current progress:  0.9999967403669695


## Some Random Test

In [ ]:
# 6295 - 1447 diff: 2.2548818588263853e-05
# 0.4030375
wd = W[:,6295].toarray().squeeze()
np.dot(xd, wd) 

0.40306005

In [ ]:
#some random test
doc_size = 10
num_samples = 11
np.random.randint(1, doc_size + 1, num_samples - 1)

array([ 1,  3,  7,  6,  3,  5,  2, 10,  5,  4])

In [ ]:
for itm in exp.local_exp:
  for idx, score in exp.local_exp[itm]:
    f_importance = score if score > 0 else 0
    f_index = x_feat_dict[txt.split()[idx]]
    # print(f_index, f_importance) # index of the feature in Xf.txt, score
    scores_all[TODO_index_instance,f_index] += f_importance

print(scores_all)

In [ ]:
# for itm,lbl in zip(test_txt,text_lbl):
txt = "walking to catch up to her people fleeing shoshoni she encounters"
lbl = [1471,7083,13199,4038]
prob_tool = Prob_wrapper(lbl)
explainer = LimeTextExplainer()
exp = explainer.explain_instance(txt, prob_tool.pred_proba, num_samples=200, num_features=1000, top_labels = len(lbl))
exp.local_exp

{0: [(2, -0.1850065399617813),
  (8, 0.177850567734265),
  (4, 0.11475629267979169),
  (1, 0.09533239002330907),
  (3, -0.09343291439751804),
  (0, 0.08113047075504802),
  (7, -0.05788686203560823),
  (6, 0.022232607830375374),
  (5, -0.01789452333922869),
  (9, 0.00944180832892001)],
 1: [(7, 0.08482325810074662),
  (4, 0.07322527739956748),
  (9, 0.06711346179032968),
  (6, 0.06710767390087848),
  (2, 0.04532743386283294),
  (3, -0.03916997918066874),
  (1, 0.022340249353170173),
  (8, 0.013145045375610365),
  (5, -0.0021419960452150116),
  (0, 0.0005242566498578924)],
 2: [(4, 0.18718297533730477),
  (7, 0.11592205600213254),
  (0, 0.0968107009115648),
  (8, 0.07767386226658486),
  (5, -0.0661630002760682),
  (9, -0.06235631322843232),
  (1, -0.057810792500153495),
  (2, -0.02757597446894796),
  (6, -0.027100377832954367),
  (3, 0.00011764991871369418)],
 3: [(6, 0.1683065213042788),
  (9, 0.1429017794016408),
  (0, 0.09609811369680278),
  (2, -0.08857669253933882),
  (4, -0.0815622

In [ ]:
true_labels = [1471,7083, 13199]
def pred_proba(sentence):
  if type(sentence) == str:
    sentence = [sentence]
  # print(sentence)
  tmp = vectorizer.predict(sentence)
  tmp.sort_indices()
  y_pred = xlm.predict(tmp) 
  # print(y_pred[:,true_labels].todense())
  return y_pred[:,true_labels].toarray()

sentence = ["walking to catch up to her people fleeing shoshoni she encounters"]
sentence_mod = ["walking to catch up to her people fleeing shoshoni she"]
corpus = sentence + sentence_mod
print(pred_proba(corpus))

[[0.4030375  0.29164279 0.29138184]
 [0.3944682  0.20462596 0.4537155 ]]


In [ ]:
explainer = LimeTextExplainer()
# text = "cherry she 1990 release english at just before"
text = "walking to catch up to her people fleeing shoshoni she encounters"
# exp = explainer.explain_instance(text, pred_proba, num_samples=200, num_features=12)
exp = explainer.explain_instance(text, pred_proba, num_samples=200, num_features=10000, top_labels = 3)

In [ ]:
exp.local_exp

{0: [(2, -0.19057729959912303),
  (8, 0.18785342663371243),
  (3, -0.1067469536264798),
  (4, 0.08911653805771073),
  (1, 0.08857007928651663),
  (0, 0.06641768957006164),
  (7, -0.0364956649974962),
  (5, -0.013020880804189745),
  (6, 0.005116026707945398),
  (9, 0.002091465596505653)],
 1: [(7, 0.08369485719086744),
  (6, 0.06288299324720106),
  (9, 0.05785127863821753),
  (4, 0.057320196487319355),
  (3, -0.04264028479271502),
  (2, 0.035640226547553805),
  (1, 0.017842465564241523),
  (5, 0.0128108430306342),
  (8, 0.010687321222354381),
  (0, 0.006541419335620768)],
 2: [(7, 0.14746150316519316),
  (4, 0.1453337592616695),
  (9, -0.09159898217894111),
  (8, 0.08710139866535505),
  (0, 0.08091909986386764),
  (2, -0.07340787599382086),
  (6, -0.0380752978942441),
  (1, -0.027243640264635497),
  (5, -0.010635117047637891),
  (3, 0.001000385863619437)]}

In [ ]:
def pred_proba(sentence):
  if type(sentence) == str:
    sentence = [sentence]
  # print(sentence)
  tmp = vectorizer.predict(sentence)
  tmp.sort_indices()
  y_pred = xlm.predict(tmp) 
  return y_pred.todense()

sentence = ["walking to catch up to her people fleeing shoshoni she encounters"]
sentence_mod = ["walking to catch up to her people fleeing shoshoni she"]
corpus = sentence + sentence_mod
print(pred_proba(corpus))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# customize my explain function to reduce time
from sklearn.linear_model import Ridge
def feature_selection(data, labels, weights, num_features = 5):
  print(1)
  clf = Ridge(alpha=0.01, fit_intercept=True, random_state=np.random.RandomState())
  clf.fit(data, labels, sample_weight=weights)
  coef = clf.coef_

  if sp.sparse.issparse(data):
      coef = sp.sparse.csr_matrix(clf.coef_)
      weighted_data = coef.multiply(data[0])
      # Note: most efficient to slice the data before reversing
      sdata = len(weighted_data.data)
      argsort_data = np.abs(weighted_data.data).argsort()
      # Edge case where data is more sparse than requested number of feature importances
      # In that case, we just pad with zero-valued features
      if sdata < num_features:
          nnz_indexes = argsort_data[::-1]
          indices = weighted_data.indices[nnz_indexes]
          num_to_pad = num_features - sdata
          indices = np.concatenate((indices, np.zeros(num_to_pad, dtype=indices.dtype)))
          indices_set = set(indices)
          pad_counter = 0
          for i in range(data.shape[1]):
              if i not in indices_set:
                  indices[pad_counter + sdata] = i
                  pad_counter += 1
                  if pad_counter >= num_to_pad:
                      break
      else:
          nnz_indexes = argsort_data[sdata - num_features:sdata][::-1]
          indices = weighted_data.indices[nnz_indexes]
      return indices
  else:
      weighted_data = coef * data[0]
      feature_weights = sorted(
          zip(range(data.shape[1]), weighted_data),
          key=lambda x: np.abs(x[1]),
          reverse=True)
      return np.array([x[0] for x in feature_weights[:num_features]])    

neighborhood_data =  np.array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
 [1., 1., 0., 0., 0., 1., 1., 1., 1., 0.],
 [0., 1., 1., 0., 1. ,0., 0., 0., 1., 1.],
 [1., 1., 1., 0., 0., 1., 0., 1., 1., 1.]])
labels_column = [0.4030375,  0.06970668, 0.5081317,  0.501127,   0.31851748]
weights = [1.00000000e+00, 3.35462628e-04, 6.66008203e-01, 5.03439617e-01,8.07800970e-01] 
feature_selection(neighborhood_data, labels_column, weights)

1


array([2, 9, 6, 0, 5])

In [ ]:
import random
ls = ['a','b','c','d','e','f','g','h']

to_delete = set(random.sample(range(len(ls)),3))
[x for i,x in enumerate(ls) if not i in to_delete]

['b', 'd', 'e', 'f', 'h']

In [ ]:
#some random test
txt = "walking to catch up to her people fleeing shoshoni she encounters"
prob_tool = Prob_wrapper([1471])
from lime.lime_text import IndexedString
ts = IndexedString(txt)
ts.num_words()
ts.raw_string()

exp = explainer.explain_instance(txt, prob_tool.pred_proba, num_samples=len(txt), num_features=20, top_labels = len(lbl))
exp.local_exp

['walking to catch up to her people fleeing shoshoni she encounters', '  catch up   people   she encounters', '  catch   her people fleeing shoshoni she ', ' to catch up to      encounters', ' to catch  to her   shoshoni  ', '  catch up   people fleeing  she ', '   up   people   she encounters', ' to catch  to  people fleeing   encounters', '          ', 'walking to catch up to her  fleeing shoshoni she encounters', 'walking to catch up to  people fleeing shoshoni she ', ' to catch  to her people  shoshoni  encounters', '  catch   her  fleeing   encounters', 'walking        shoshoni  ', 'walking to   to her people fleeing shoshoni she encounters', ' to   to her    she ', '          ', ' to   to her   shoshoni  ', ' to  up to  people fleeing   ', '   up      she encounters', 'walking          ', '       fleeing   encounters', '          ', '  catch up  her people   she encounters', '          ', '   up  her   shoshoni  ', '          ', 'walking   up       ', ' to   to   fleeing   encoun

{0: [(2, -0.17390160855285747),
  (8, 0.1695847553736939),
  (1, 0.1148756549316466),
  (4, 0.10278103020446847),
  (3, -0.10153039659923374),
  (0, 0.08973654364769076),
  (7, -0.07244112964211905),
  (9, 0.010123238725934356),
  (6, 0.010097958204189036),
  (5, -0.008909512009377946)]}

In [ ]:
# some random test
import scipy
import numpy as np
W1 = scipy.sparse.load_npz("/content/260CDL_22Winter/model/ranker/0.model/W.npz")
W2 = scipy.sparse.load_npz("/content/260CDL_22Winter/model/ranker/1.model/W.npz")
W3 = scipy.sparse.load_npz("/content/260CDL_22Winter/model/ranker/2.model/W.npz")
# print(W[:,1471].shape)
x_csr = vectorizer.predict(["walking to catch up to her people fleeing shoshoni she encounters"])
print(x_csr.shape)
print(xlm.predict(x_csr))
# print(type(x_csr), type(W[:,1471]))

xd = x_csr.toarray().squeeze()
xd = np.append(xd, 1.0)
# xd = np.insert(xd, 0, 1.0)
base = 100
idx = -1
wcnt = 0
for W, rg in zip([W1, W2, W3],[16, 256, 13330]):
  for i in range(0, rg):
    wd = W[:,i].toarray().squeeze()
    rst = abs(np.dot(xd, wd) - 0.29164279)
    if rst  < base:
      base = rst
      idx = i
  print("done", rg)
  wcnt += 1
print(base)
print(idx)
print(wcnt)

# C = scipy.sparse.load_npz("/content/260CDL_22Winter/model/ranker/2.model/C.npz")
# C
# 0.0002913217045769634

(1, 910300)
  (0, 1471)	0.4030375
  (0, 7083)	0.29164279
  (0, 13199)	0.29138184
  (0, 4038)	0.26829728
  (0, 11235)	0.18322547
  (0, 7892)	0.17281087
  (0, 8108)	0.06291683
  (0, 12630)	0.06291683
  (0, 12851)	0.035093043
  (0, 5879)	0.034503333
  (0, 1274)	0.030859185
  (0, 719)	0.028692868
  (0, 10185)	0.018604755
  (0, 4921)	0.012175326
  (0, 7891)	0.010081249
  (0, 4923)	0.009986753
  (0, 12583)	0.008487523
  (0, 12341)	0.0075502424
  (0, 12505)	0.007487659
  (0, 7719)	0.007457245
done 16
done 256
done 13330
0.008126493310465832
5955
3
